### **Toulouse School of Economics**
#### **M2 Statistics & Econometrics**
---
## **Web Mining**
### Final Project
---
#### Anh-Dung LE
#### Sherman ALINE
---

## Import necessary packages and setting for multiprocessing 

In [2]:
import pandas as pd
import numpy as np
import string, unidecode, re, os, json, time, scipy.sparse
import matplotlib as plt
import networkx as nx
from networkx.algorithms.traversal.depth_first_search import dfs_tree
from dask.distributed import Client
import dask.dataframe as dd
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer as stemmer
from multiprocessing import dummy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold

core = os.cpu_count() - 1
client = Client(n_workers = core, threads_per_worker = 1, memory_limit = '25GB')
P = dummy.Pool(processes = core)

## Import dataset

In [ ]:
path = 'E:\\M2 EconStat\\Web Mining\\Project\\comments_students.csv'
df = pd.read_csv(path, header = 0, engine = 'c',
                 usecols = ['created_utc', 'ups', 'link_id', 'id', 'author', 'body', 'parent_id'])

# We remove prefix so that columns "parent_id" and "link_id" have the same format as column "id"
df['parent_id'] = df['parent_id'].str.split('_', expand = True)[1]
df['link_id'] = df['link_id'].str.split('_', expand = True)[1]

# We save this dataset for later use
path = r'E:\\M2 EconStat\\Web Mining\\prefix_removed.csv'
df.to_csv(path, index = False, encoding =  'utf-8')

## Find the root for every spanning tree

In [ ]:
path = r'E:\\M2 EconStat\\Web Mining\\prefix_removed.csv'
df = pd.read_csv(path, header = 0, engine = 'c')

If we work with subgraphs characterized by `link_id`, then some features (for example, depth of a node w.r.t the *root*) can not be computed because a subgraph possibly contains sub-subgraphs that are disconnected from each others. That's why we refine the definition of a *root*. The idea is that a comment is a root if it does not reply any comment in the dataset. Hence its `in_degree` is $0$. Of course, a comment with `link_id` = `parent_id` is certainly a root.

In [18]:
G = nx.from_pandas_edgelist(df, source = 'parent_id', target = 'id', create_using = nx.DiGraph())
roots = [n for n, d in G.in_degree() if d == 0]

Function `para_func` below takes a root as input and returns a dataframe. Column `id` contains all the nodes of the tree characterized by the root. We extract this tree by `dfs_tree`. All elements in column `root` are the same, i.e the root itself. 

In [20]:
def para_func(r):
    node = list(dfs_tree(G, r).nodes)
    tree = G.subgraph(node)
    node.remove(r)
    root =  pd.DataFrame({'id': node, 'root': [r] * len(node)})
    return root

This operation is not computationally intensive, so I use `multiprocessing.dummy.Pool` for simple implementation.

In [ ]:
root_node_list = P.map(para_func, roots)
root_node = pd.concat(root_node_list)

We add this root-node information to original dataframe. Notice that the original dataframe does not contain comments that are roots. That's why we specify `how = 'left'`.

In [ ]:
df = df.merge(root_node, how = 'left', on = 'id')

# We save this dataset for later use
path = r'E:\\M2 EconStat\\Web Mining\\root_included.csv'
df.to_csv(path, index = False, encoding =  'utf-8')

## Motivation for the choice of tree-related features

In this section,

- I use the words "node" and "comment" interchangably.

- If not stated otherwise, I meant by "the tree" the one characterized by the **root** of the comment.

Then

- `depth`: the shortest path from its root to the comment.

If the `depth` is too large, the comments are less likely to be seen by other users, and thus less likely to be upvoted or downvoted.

- `num_siblings`: the number of nodes within the tree that share the same parent.

- `num_comments_author`: the number of comments made by the author within the tree. If the author made more than $1$ comments within the tree, then these comments have the same value of `num_comments_author`.

- `num_previous_comments`: the number of nodes whose posting time is stricly sooner than that of the comment.

- `num_later_comments`: the number of nodes whose posting time is stricly later than that of the comment.

I add `num_siblings`, `num_comments_author`, `num_previous_comments`, and `num_later_comments` because I have seen them in the paper [Conversation Modeling on Reddit Using a Graph-Structured LSTM](https://www.aclweb.org/anthology/Q18-1009.pdf).

- `time_since_root`: the interval of posting time between the comment and its **root**.

- `time_since_parent`: the interval of posting time between the comment and its direct parent.

If `time_since_root` or `time_since_parent` is small, then the comment are more likely to be seen by later users, and thus more likely to be upvoted or downvoted.

- `num_comments_subtree`: the number of nodes in the tree.

- `height_subtree`: the longest (directed) path in the tree.

- `num_children`: the number of **direct** replies to the comment.

`num_comments_subtree`, `height_subtree`, and `num_children` are size proxies of the tree rooted from the comment. It seems that interesting comments attracts more replies and thus its induced tree has bigger size.

## Extract tree-related features

In [2]:
path = r'E:\\M2 EconStat\\Web Mining\\root_included.csv'
df = dd.read_csv(path, header = 0)

# We create column "timestamp" to compute time interval later
df['timestamp'] = dd.to_datetime(df['created_utc'], utc = True, unit = 's')

I could not find any package containing functions to compute all features, except for *depth of the comment*, *height of the subtree*, and *size of that subtree*. Luckily, our trees are of a special kind, [arborescence](https://www.wikiwand.com/en/Tree_(graph_theory)#/Rooted_tree). This allows us to utilize package `dask` and highly optimized functions from package `pandas` to speed up computation. First, we compute features related to the tree characterized by column `root`. Comments with the same `root` are in the same tree.

In [4]:
def features_extract(sub_df):
    r = sub_df['root'].iloc[0]
    tree = nx.from_pandas_edgelist(sub_df,
                                   source = 'parent_id',
                                   target = 'id',
                                   create_using = nx.DiGraph())

    count_utc = sub_df.groupby('created_utc').size()
    cum_previous_counts = count_utc.sort_index(ascending = True).shift(fill_value = 0).cumsum()
    cum_later_counts = count_utc.sort_index(ascending = False).shift(fill_value = 0).cumsum()
    time_parent = sub_df.parent_id.map(dict(zip(sub_df.id, sub_df.timestamp)))

    depth = [nx.shortest_path_length(tree, source = r, target = n) for n in sub_df.id]
    num_siblings = sub_df.groupby('parent_id').parent_id.transform('count')
    num_children = sub_df.groupby('parent_id').size().reindex(sub_df.id, fill_value = 0)
    num_comments_author = sub_df.groupby('author').author.transform('count')
    num_previous_comments = sub_df.created_utc.map(cum_previous_counts)
    num_later_comments = sub_df.created_utc.map(cum_later_counts)
    time_since_root = (sub_df.timestamp - sub_df.timestamp.min()) / pd.Timedelta(hours = 1)
    time_since_parent = ((sub_df.timestamp - time_parent) / pd.Timedelta(hours = 1)).fillna(0)

    data = list(zip(sub_df.id, depth, num_siblings, num_children, num_comments_author,
                    num_previous_comments, num_later_comments, time_since_root, time_since_parent))

    columns = ['id', 'depth', 'num_siblings', 'num_children', 'num_comments_author',
              'num_previous_comments', 'num_later_comments', 'time_since_root', 'time_since_parent']

    return pd.DataFrame(data, columns = columns)

meta = {'id': object, 'depth': np.int64, 'num_siblings': np.int64, 'num_children': np.int64,
       'num_comments_author': np.int64, 'num_previous_comments': np.int64, 'num_later_comments': np.int64,
       'time_since_root': np.float64, 'time_since_parent': np.float64}

result = df.groupby('root').apply(features_extract, meta = meta)
result = result.compute(scheduler = 'processes')
result.reset_index(inplace = True, drop = True)

In [5]:
result

,id,depth,num_siblings,num_children,num_comments_author,num_previous_comments,num_later_comments,time_since_root,time_since_parent
0,cr1qq1w,1,1,0,1,0,0,0.000000,0.000000
1,cqvdyoy,1,1,0,1,0,0,0.000000,0.000000
2,cqxivp8,1,1,0,1,0,0,0.000000,0.000000
3,cqxiruo,1,1,0,1,0,0,0.000000,0.000000
4,cqws27u,1,1,0,1,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
4234965,cquiajn,1,1,1,1,0,2,0.000000,0.000000
4234966,cqumskx,2,1,1,1,1,1,2.025556,2.025556
4234967,cquxm86,3,1,0,1,2,0,11.754444,9.728889
4234968,cqugv6d,1,1,0,1,0,0,0.000000,0.000000


*Remark 1:*

- I use `.compute()` or equivalently `.compute(scheduler = 'threads')`. The executing time is reduced to $30$ minutes. The utilization of CPU increases from $25\%$ to $50\%$. Previously, I used `multiprocessing.dummy.Pool()` and my laptop could not finish the computation in 3 hours. I had no choice but to interupt Python kernel.

- Then I come across this [answer](https://stackoverflow.com/a/31364127/7357673) and change to `.compute(scheduler = 'processes')`. The utilization of CPU is almost $100\%$. The executing time is reduced to just $12$ minutes. This is awesome.

*Remark 2:*

- At first I thought `dask` only works with function that returns a dictionary. This means I have to do $2$ more steps, i.e. converting resulted dictionaries to pandas dataframe and then concatenating them together. Even I used `multiprocessing.dummy.Pool()` for the conversion, it still took up to $2.5$ minutes.

- Then I come across this [documentation](https://docs.dask.org/en/latest/dataframe-design.html#metadata). It tells that `dask` works perfectly with function that returns a pandas dataframe (It should ^_^). We only need to specify the correct data types with parameter `meta`. One advantage of this approach is that `dask` automatically concatenate resulted dataframes.

*Remark 3:*

- Because we have no information about the root, except for its `id`, we can not compute exactly `time_since_root`. Similarly, if the parent of a comment is itself a root, then `time_since_parent` can not be computed for this comment. Our solution is to take the earliest available posting time within a tree as a proxy for posting time of the root.

## Number of comments and height of the subtree rooted from the comment itself

I first create a graph from the whole dataset.

In [6]:
G = nx.from_pandas_edgelist(df,
                            source = 'parent_id',
                            target = 'id',
                            create_using = nx.DiGraph())

In this case, `multiprocessing.dummy.Pool` and `dask.Series.apply` take almost the same amount of time (about $3$ minutes and $45$ seconds) to finish. This makes sense because `features_extract` is applied on **every** comment in the dataset. Hence `dask.Series.apply` does not have any advantage by cleverly partitioning the dataframe. Last but not least, `dask.Series.apply` has a disadvantage, i.e. it can not be called on an **existing** Dask dataframe. To use it, we need to import our dataset again with `pd.read_csv`.

In [7]:
def features_extract(r):
    subtree = dfs_tree(G, r)
    num_comment = subtree.number_of_nodes()
    height_subtree = nx.dag_longest_path_length(subtree)
    return [r, num_comment, height_subtree]

result2 = P.map(features_extract, df.id)
result2 = pd.DataFrame(result2, columns = ['id', 'num_comments_subtree', 'height_subtree'])
result2

,id,num_comments_subtree,height_subtree
0,cqug90j,1,0
1,cqug90k,1,0
2,cqug90z,1,0
3,cqug91c,1,0
4,cqug91e,3,2
...,...,...,...
4234965,crrbelu,1,0
4234966,crrbelv,1,0
4234967,crrbemp,1,0
4234968,crrbenh,1,0


We add these features to our original daraframe. By construction, `df`, `compute_result`, and `sub_tree_infor` have exactly the same elements in column `id`. That's why dont need to specify parameter `how`.

In [8]:
path = r'E:\\M2 EconStat\\Web Mining\\root_included.csv'
df = pd.read_csv(path, header = 0, engine = 'c')
df = df.merge(result, on = 'id')
df = df.merge(result2, on = 'id')
df

,created_utc,ups,link_id,id,author,body,parent_id,root,depth,num_siblings,num_children,num_comments_author,num_previous_comments,num_later_comments,time_since_root,time_since_parent,num_comments_subtree,height_subtree
0,1430438400,3.0,34f9rh,cqug90j,jesse9o3,No one has a European accent either because i...,cqug2sr,cqug2sr,1,1,0,1,0,0,0.000000,0.000000,1,0
1,1430438400,3.0,34fvry,cqug90k,beltfedshooter,That the kid ..reminds me of Kevin. so sad :-(,34fvry,34fvry,1,1443,0,1,0,3512,0.000000,0.000000,1,0
2,1430438400,5.0,34ffo5,cqug90z,InterimFatGuy,NSFL,cqu80zb,cqu80zb,1,5,0,1,0,11,0.000000,0.000000,1,0
3,1430438401,1.0,34aqsn,cqug91c,JuanTutrego,I'm a guy and I had no idea this was a thing g...,cqtdj4m,cqtdj4m,1,4,0,1,0,4,0.000000,0.000000,1,0
4,1430438401,101.0,34f9rh,cqug91e,dcblackbelt,"Mid twenties male rocking skinny jeans/pants, ...",cquc4rc,cquc4rc,1,2,1,1,0,3,0.000000,0.000000,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234965,1433116795,NaN,37y5rx,crrbelu,monster860,Does the sun set in the west/rise in the east?...,37y5rx,37y5rx,1,1496,0,1,3265,0,12.452222,0.000000,1,0
4234966,1433116795,NaN,37y5rx,crrbelv,jsimo36,Coffee.,37y5rx,37y5rx,1,1496,0,1,3265,0,12.452222,0.000000,1,0
4234967,1433116796,NaN,380jx2,crrbemp,torianicole731,"people who cannot make up their mind, my bulls...",380jx2,380jx2,1,9,0,1,8,0,0.522500,0.000000,1,0
4234968,1433116797,NaN,37yawp,crrbenh,bellypouch,Give them to Irish people in exchange for doin...,37yawp,37yawp,1,1779,0,1,3519,0,11.398056,0.000000,1,0


In [9]:
# We save it for later use
path = r'E:\\M2 EconStat\\Web Mining\\features_included.csv'
df.to_csv(path, index = False, encoding =  'utf-8')

## Text features

As a user of reddit, I will apply some domain knowledge in the process of selecting text features. 

- Emojis - emoticons may be used to detect sentiment. Additionally, they may signal a overly-casual post which may not be recieved well.

- Sentiment in post - funny comments, as well as serious and informative posts will get upvotes. In long comment threads, a hostile tone may get upvotes (for one party, while the other party with hostile tone will get many downvotes).

- Length - long comments may be an indication of high-information or high-effort post.

- #Punctuation - this is another indicator that a post is likely to be well-written, correlated with useful information and understandable writing etc. All things which will get upvotes on AskReddit. Extremely large amounts of punctuation may also indicate visual text jokes being made using markdown.

- #Paragraphs - indicator of high amiount of content/contribution or high effort. We detect paragraphs by the number of double spaces. Doublespaces are removed on reddit. Through manual investigation, I have determined that double spaces in our corpora are new lines.

- Capital letters.

- Number of links in a post - citations generally get upvotes, an indicator of sharing information.

- Refer to other users `/u/*`.

- Refer to subreditt `/r/`.

- Quotes - detect, count number of them, remove.

## Build Features

In [42]:
path = r'E:\\M2 EconStat\\Web Mining\\features_included.csv'
df = pd.read_csv(path, header = 0, engine = 'c')
ind = (df['body'] != 'deleted') & (df['body'] != '[deleted]') & df['body'].notna()

`ind` is the indices of  "normal" rows that will be fed into the model later. Next we define some strings used in formatting.

In [43]:
newline = '  '
lessthan = '&lt;'
greaterthan = '&gt;'
emojis = '[\\u203C-\\u3299\\U0001F000-\\U0001F644]'
hyperlink = '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

All manually added text features have the prefix `txt_`.

In [44]:
# Count then remove the number of markdown links
df['txt_md_link_count'] = df.body.str.count('\[[^\]]+\]\([^\)]+\)')
df['body'] = df.body.str.replace('\[(?P<txt>[^\]]+)\]\([^\)]+\)', '\g<txt>')

# Count then remove the number of links with no markdown
df['txt_nomk_link_count'] = df.body.str.count(hyperlink)
df['body'] = df.body.str.replace(hyperlink, '') 

# Reddit links: count then remove the /r/ part but keep the name of sub
df['txt_reddit_links'] = df.body.str.count('/r/')
df['body'] = df.body.str.replace('/r/', '')

# User links 
df['txt_reddit_links'] = df.body.str.count('/u/')
df['body'] = df.body.str.replace('/u/', '')

# Count quotes and remove them
df['txt_num_quotes'] = df.body.str.count(greaterthan+'(.+)'+newline)
df['body'] = df.body.str.replace(greaterthan+'(.+)'+newline, 'QQUUOOTTEE ') # We use custom token and replcae w e.g. $$QQUUOOTTEE$$

# Count emojis in each comment and remove them
df['txt_num_emoji'] = df.body.str.count(emojis)
df['body'] = df.body.str.replace('('+emojis+')', '')

# Count capitals 
df['txt_num_caps'] = df.body.str.count('[A-Z]') # Signal good grammar

# Comment length
df['txt_length'] = df.body.str.len() # Relate to count capitals

# Exclamation marks
df['txt_excmarks'] = df.body.str.count('\!') # People usually overuse

# Question marks
df['txt_qmarks'] = df.body.str.count('\?') # People usually overuse

# Other common punctuation.
df['txt_punct'] = df.body.str.count('[\-\"\'\,]') # The post is well-written

# Number related marks
df['txt_mathmarks'] = df.body.str.count('[\$\%\+\=]') # For comments more technical

# Numbers
df['txt_numerals'] = df.body.str.count('\d') # For comments more technical

## Text cleaning

In [ ]:
def tokenize_stemming(x):
    x = x.split()
    x = [stemmer().stem(w) for w in x if w not in my_stopwords]
    x = ' '.join(x)
    return x

tmp = df['body'][ind].copy()

# Lowercase
tmp = tmp.str.lower()

# Delete punctuation
tmp = tmp.str.replace('[\\n\\r\/\-]', ' ')
tmp = tmp.str.replace(r"((?!{}).)".format('(\\b[-/]\\b|[a-zA-Z0-9])'), ' ', regex = True)

#Tokenize and stemming
tmp = P.map(tokenize_stemming, tmp)

body = df.body.copy()
body[ind] = tmp
df['body'] = body

# We save it for later use
path = r'E:\\M2 EconStat\\Web Mining\\stemming.csv'
df.to_csv(path, index = False, encoding =  'utf-8')

## TF-IDF

Our idea is as follows. In order to disregard words which are commonly used, we first generate TF-IDF and then drop features with low variance. We create a large set of features and then use feature selection to shrink it to a manageable number.

Before stemming: no terms with 0.2 min df exist. We choose max_df = 0.01 because it is the highest df which removes grammar words such as i've etc, leaving more unique words.

After stemming: extremely aggressive stemming algorithnm (many words no longer human readable)
but still: no terms with 0.2 min df exist.

Through manual investigation of words and document frequency,
we decide on a min_df of __ and a max_df of ___

We see that for above the max_df > 0.018, there were common words with little signal such as
kind yeah play best talk thought etc... (check at max_df=0.8, display 800:850 of 1000 features

For below the min_df < 1400, there were many proper nouns and slang such as bieber, meow, morrowind, obama, yooooo, and hubby, runescape

With these constaints, we only get 3001 features, so we loosen them a bit and then remove irrelevant ones later with feature selection techniques.

In [51]:
path = r'E:\\M2 EconStat\\Web Mining\\stemming.csv'
df = pd.read_csv(path, header = 0, engine = 'c')

n_feature = 5000
tfi_df_vec = TfidfVectorizer(use_idf = True,
                             min_df = 0.00011,
                             max_df = 0.018, 
                             stop_words = 'english',
                             max_features = n_feature)

data = tfi_df_vec.fit_transform(df['body'].fillna(''))
thresh = 0.001
sel = VarianceThreshold(threshold = thresh)
features = sel.fit_transform(data)
features = pd.DataFrame.sparse.from_spmatrix(features)
df2 = pd.concat([df, features], axis = 1)

## Split dataset into test and train sets

In [52]:
ind_test  = df.ups.isna()
ind_train = ~ ind_test
X = df2.iloc[:, 8:].fillna(0)
y = df.ups
X_train, y_train = X.loc[ind_train, :], y[ind_train]
X_test, y_test = X.loc[ind_test, :], y[ind_test]

In [53]:
y_test_full = df.ups

## Subsampling

Because training set contains over $3$ millions rows and thus takes too much time to run, I create a subsample whose size is $10\%$ of the original training set. To make this subsample more representative, I divide the training set into $100$ categories. The $i$-th category contains comments whose the percentile of their `ups` belong to the interval $[i, i+1]$. For each category, we randomly select $10\%$ of the total comments in that category.

We use function `pandas.qcut` to get the percentile (w.r.t `ups`) of each comment. It's mentioned in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html):

>q: Number of quantiles. 10 for deciles, 4 for quartiles, etc. Alternately array of quantiles, e.g. [0, .25, .5, .75, 1.] for quartiles.

So to get percentile, we should set `q` equal to $100$. However, setting `q = 100` does not do what we expect. We still don't know why. With trial and error, we found that `q = 1710` produces $100$ percentile as expected.

In [54]:
percentile = pd.qcut(y_train, q = 1710, labels = False, duplicates = 'drop')
ind_sub = []
prop = 0.1
for i in np.unique(percentile):
    ind_tmp = percentile == i
    ind_sub = ind_sub + pd.Series(y_train[ind_tmp].index).sample(frac = prop).tolist()
sub_X_train, sub_y_train = X_train.iloc[ind_sub], y_train.iloc[ind_sub]

## Random forest model

In [1]:
model = RandomForestRegressor(n_jobs = -1)
model.fit(sub_X_train, sub_y_train)

# Get the mean absolute error on the validation data
y_pred = model.predict(X_test)
MAE = mean_absolute_error(y_pred , y_test_full)
print('Random forest validation MAE = ', MAE)

## XGBoost model

In [2]:
XGBModel = XGBRegressor()
XGBModel.fit(sub_X_train, sub_y_train, verbose = False)

# Get the mean absolute error on the validation data :
y_pred2 = XGBModel.predict(X_test)
MAE = mean_absolute_error(y_pred2, y_test_full)
print('XGBoost validation MAE = ', MAE)

## lightGBM model

In [4]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(sub_X_train, sub_y_train)
# Get the mean absolute error on the validation data
y_pred3 = model.predict(X_test)
MAE = mean_absolute_error(y_pred3, y_test_full)
print('Random forest validation MAE = ', MAE)

In [58]:
# path = r'E:\\M2 EconStat\\Web Mining\\prediction.csv'
# prediction = pd.DataFrame({'id': df.loc[ind_test, 'id'], 'predicted': y_pred3})
# prediction.to_csv(path, index = False, encoding =  'utf-8', header = True)